In [1]:
import json
import os

from nomad_auto_xrd.auto_xrd_analysis import run_analysis

# Load the NOMAD archive data
with open('model_metadata.archive.json') as file:
    archive_data = json.load(file)

# Extract CIF files and XRD model path from the archive
reference_cifs = archive_data['data']['cif_files']
xrd_model_file = archive_data['data']['xrd_model_file']

# Load the diffraction pattern archive
with open('xrd_archive.json') as file:
    xrd_archive = json.load(file)

# Get the diffraction patterns from the NOMAD archive
patterns = xrd_archive['archive']['results']['properties']['structural'][
    'diffraction_pattern'
]

# Ensure temp_backup directory exists to store the spectrum files
if not os.path.exists('temp_patterns'):
    os.mkdir('temp_patterns')

# Write the spectrum file from the patterns data
for i, pattern in enumerate(patterns):
    with open(f'temp_patterns/spectrum_{i}.xy', 'w') as f:
        two_theta_angles = pattern['two_theta_angles']
        intensities = pattern['intensity']
        for angle, intensity in zip(two_theta_angles, intensities):
            f.write(f'{angle} {intensity}\n')


def get_min_max_angles(patterns):
    all_angles = []
    for pattern in patterns:
        all_angles.extend(pattern['two_theta_angles'])
    return min(all_angles), max(all_angles)


# Example usage (assuming you have already loaded the NOMAD archive data)
min_angle, max_angle = get_min_max_angles(patterns)

# Run the analysis
run_analysis(
    references_folder='References',  # Ensure your reference CIF files are in this folder  # noqa: E501
    spectra_folder='temp_patterns',  # This is where you stored spectra
    xrd_model_path=xrd_model_file,  # Path to your XRD model
    results_file='custom_results.json',  # Output file for results
    max_phases=5,  # Adjust phases as needed
    min_conf=30,  # Confidence threshold
    inc_pdf=False,
    min_angle=min_angle,
    max_angle=max_angle,  # No PDF model
)